In [2]:
import nfl_data_py as nfl
import numpy as np
import pandas as pd
import ssl
import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import pipeline
# import tensorflow as tf
import torch
import shap
import transformers

import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)
ssl._create_default_https_context = ssl._create_unverified_context


In [3]:
years = list(np.arange(2010, 2024))

weekly_raw = nfl.import_weekly_data(
    years,
    downcast=True
)

weekly_raw.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000865,None,Charlie Batch,QB,QB,None,PIT,2010,2,REG,TEN,5,11,25.0,0,0.0,2.0,6.0,0,0,110.0,21.0,0.0,-10.005162,0,0.227273,-0.015931,1,0.0,0,1.0,0.0,0.0,-0.982634,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,1.000000,1.000000
1,00-0000865,None,Charlie Batch,QB,QB,None,PIT,2010,3,REG,TB,12,17,186.0,3,2.0,0.0,-0.0,0,0,153.0,55.0,7.0,7.614506,0,1.215686,0.332648,5,26.0,0,0.0,0.0,2.0,3.388114,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,18.040001,18.040001
2,00-0000865,None,Charlie Batch,QB,QB,None,PIT,2010,4,REG,BAL,12,21,141.0,0,1.0,2.0,15.0,0,0,278.0,46.0,6.0,-0.152696,0,0.507194,0.053022,1,4.0,0,0.0,0.0,0.0,-0.714745,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,4.040000,4.040000
3,00-0001231,None,Josh Bidwell,P,SPEC,None,WAS,2010,1,REG,DAL,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,1,0.0,0,1.0,0.0,0.0,-3.118365,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000
4,00-0001545,None,Todd Bouman,QB,QB,None,JAX,2010,7,REG,KC,18,34,222.0,2,2.0,1.0,6.0,0,0,250.0,121.0,12.0,-5.270316,0,0.888000,0.078782,2,8.0,0,0.0,0.0,1.0,2.377490,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.680000,13.680000


In [4]:
weekly_sum_cols = ['position', 'completions', 'attempts', 'passing_yards', 'passing_tds',
                   'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
                   'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
                   'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
                   'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
                   'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
                   'receptions', 'targets', 'receiving_yards', 'receiving_tds',
                   'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
                   'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
                   'receiving_2pt_conversions', 'racr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']

weekly_sum = weekly_raw.groupby(['recent_team', 'season', 'week']).sum()[weekly_sum_cols].reset_index()
weekly_sum


,recent_team,season,week,position,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,ARI,2010,1,WRTEQBWRWRRBRBRBWRWR,22,41,297.0,1,0.0,2.0,31.0,2,1,431.0,119.0,13.0,5.936007,0,0.689095,0.104602,21,112.0,1,2.0,2.0,6.0,-10.699470,0,22,40,297.0,1,1.0,1.0,424.0,119.0,13.0,7.608192,0,18.759920,0.0,60.779999,82.779999
1,ARI,2010,2,RBWRQBWRTERBRBQBWRWR,18,33,164.0,0,3.0,2.0,15.0,1,0,362.0,76.0,8.0,-18.103937,0,0.601331,-0.009175,13,118.0,1,0.0,0.0,2.0,3.129859,0,18,32,164.0,0,0.0,0.0,345.0,76.0,8.0,-14.279747,0,8.280849,0.0,34.759998,52.760002
2,ARI,2010,3,RBWRQBWRTERBRBRBWRWR,12,26,122.0,2,1.0,2.0,14.0,0,0,235.0,43.0,7.0,-6.220401,0,0.519149,-0.029862,26,119.0,0,0.0,0.0,7.0,-2.145487,0,12,25,122.0,2,0.0,0.0,228.0,43.0,7.0,-2.323832,0,6.186080,1.0,52.980000,64.980003
3,ARI,2010,4,WRQBTERBRBRBQBWRWRTE,15,28,146.0,0,2.0,9.0,69.0,1,1,231.0,28.0,7.0,-24.353945,0,1.292399,-0.008910,14,47.0,0,0.0,0.0,3.0,-0.790223,0,15,27,146.0,0,0.0,0.0,231.0,28.0,7.0,-5.261675,0,5.394284,0.0,19.139999,34.139999
4,ARI,2010,5,WRTEQBFBTERBRBRBQBWRWRWRTE,18,29,174.0,0,1.0,4.0,21.0,2,0,223.0,58.0,8.0,-0.924068,0,1.337907,0.031181,24,41.0,0,1.0,0.0,1.0,-9.792395,0,18,29,174.0,0,1.0,0.0,223.0,58.0,8.0,-0.006318,0,7.061762,0.0,26.459999,44.459999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7581,WAS,2023,13,TEWRWRRBWRTEQBTEWRRBRB,12,23,127.0,0,1.0,3.0,20.0,0,0,198.0,43.0,5.0,-14.074273,1,0.641414,-0.021260,28,138.0,2,0.0,0.0,6.0,3.306135,0,12,22,127.0,0,1.0,0.0,171.0,43.0,5.0,-8.482146,1,11.001282,0.0,45.580002,57.580002
7582,WAS,2023,15,PTERBQBWRWRWRRBWRTEQBWRRB,19,36,226.0,3,1.0,1.0,8.0,0,0,364.0,54.0,9.0,-3.150248,0,1.528847,0.378468,21,79.0,0,0.0,0.0,3.0,-5.227423,0,19,34,226.0,3,0.0,0.0,347.0,54.0,9.0,0.539493,0,23.446604,0.0,67.540001,86.540001
7583,WAS,2023,16,TEWRQBWRWRWRRBTEQBWRRB,16,35,156.0,1,2.0,1.0,13.0,0,0,197.0,101.0,7.0,-16.542477,0,1.904697,0.147889,22,102.0,3,0.0,0.0,7.0,5.943290,0,16,32,156.0,1,0.0,0.0,147.0,101.0,7.0,-11.606894,0,21.396389,0.0,56.040001,72.040001
7584,WAS,2023,17,TEWRWRWRRBTEQBWRRB,17,28,169.0,1,2.0,1.0,6.0,0,0,172.0,103.0,6.0,-9.632565,0,0.982558,0.025758,14,62.0,0,0.0,0.0,5.0,1.756041,0,17,26,169.0,1,0.0,0.0,146.0,103.0,6.0,-5.751150,0,-27.317482,0.0,35.860001,52.860001


In [5]:
scores_raw = pd.read_csv('spreadspoke_scores.csv')

scores_raw.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [6]:
name_id_dict = {
    "Arizona Cardinals": "ARI",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC",
    "Las Vegas Raiders": "LV",
    "Los Angeles Chargers": "LAC",
    "Los Angeles Rams": "LAR",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NE",
    "New Orleans Saints": "NO",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN",
    "Washington Commanders": "WAS"
}

scores = scores_raw[(scores_raw['schedule_season'] >= 2010)
                    & (scores_raw['schedule_playoff'] == False)][['team_home', 'score_home', 'team_away', 'score_away',
                                                                  'schedule_season', 'schedule_week']]

scores['team_home'] = scores['team_home'].replace(name_id_dict)
scores['team_away'] = scores['team_away'].replace(name_id_dict)
scores['schedule_week'] = scores['schedule_week'].astype('int32')
scores

,team_home,score_home,team_away,score_away,schedule_season,schedule_week
10008,NO,14,MIN,9,2010,1
10009,BUF,10,MIA,15,2010,1
10010,CHI,19,DET,14,2010,1
10011,HOU,34,IND,24,2010,1
10012,JAX,24,DEN,17,2010,1
...,...,...,...,...,...,...
13783,NO,48,ATL,17,2023,18
13784,NYG,27,PHI,10,2023,18
13785,SF,20,LAR,21,2023,18
13786,TEN,28,JAX,20,2023,18


In [7]:
team_scores = pd.DataFrame()

for i, row in scores.iterrows():
    season = row['schedule_season']
    week = row['schedule_week']
    team = row['team_home']
    score = row['score_home']

    team_scores_temp = pd.DataFrame([{'season': season, 'week': week, 'team': team, 'score': score, 'is_home': 1}])
    team_scores = pd.concat([team_scores, team_scores_temp])

    team = row['team_away']
    score = row['score_away']

    team_scores_temp = pd.DataFrame([{'season': season, 'week': week, 'team': team, 'score': score, 'is_home': 0}])
    team_scores = pd.concat([team_scores, team_scores_temp])

team_scores

,season,week,team,score,is_home
0,2010,1,NO,14,1
0,2010,1,MIN,9,0
0,2010,1,BUF,10,1
0,2010,1,MIA,15,0
0,2010,1,CHI,19,1
...,...,...,...,...,...
0,2023,18,LAR,21,0
0,2023,18,TEN,28,1
0,2023,18,JAX,20,0
0,2023,18,WAS,10,1


In [8]:
weeks = team_scores.merge(weekly_sum,
                       how='inner',
                       right_on=['recent_team', 'season', 'week'],
                       left_on=['team', 'season', 'week']).drop(columns=['recent_team'])


weeks

,season,week,team,score,is_home,position,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,2010,1,NO,14,1,FBQBTEWRWRRBTEWRRBWR,27,36,237.0,1,0.0,1.0,8.0,1,0,258.0,124.0,12.0,1.133939,0,0.918605,0.098923,25,79.0,1,0.0,0.0,6.0,-0.422439,0,27,35,237.0,1,0.0,0.0,247.0,124.0,12.0,3.699585,0,9.483550,0.0,57.080002,84.080002
1,2010,1,MIN,9,0,QBTEWRTETEWRWRRBRBWR,15,27,171.0,1,1.0,1.0,9.0,0,0,271.0,73.0,7.0,-4.680712,0,0.630996,-0.009404,23,91.0,0,0.0,0.0,5.0,-1.493683,0,15,27,171.0,1,0.0,0.0,271.0,73.0,7.0,-2.608740,0,-8.596407,0.0,41.040001,56.040001
2,2010,1,BUF,10,1,PWRWRRBRBQBWRWRRB,18,34,139.0,1,0.0,3.0,23.0,0,0,220.0,44.0,6.0,-7.514690,0,0.631818,0.002088,17,50.0,0,0.0,0.0,3.0,-4.570020,0,18,31,139.0,1,0.0,0.0,213.0,44.0,6.0,0.738895,0,2.092527,0.0,34.459999,52.459999
3,2010,1,MIA,15,0,RBFBRBTEWRWRQBWRWR,21,34,182.0,0,0.0,3.0,18.0,0,0,219.0,84.0,9.0,-0.546047,0,0.831050,0.024401,36,132.0,1,0.0,0.0,10.0,0.514395,0,21,34,182.0,0,0.0,0.0,219.0,84.0,9.0,3.211347,0,-4.741334,0.0,44.680000,65.680000
4,2010,1,CHI,19,1,RBQBWRWRTERBWR,23,35,372.0,2,1.0,4.0,10.0,1,1,330.0,212.0,15.0,5.098709,0,1.127273,0.146324,31,101.0,0,0.0,0.0,4.0,-6.460926,0,23,35,372.0,2,3.0,2.0,330.0,212.0,15.0,3.508837,0,11.271830,0.0,74.180000,97.179993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2023,18,SF,20,1,WRWRQBWRWRTERBRBWR,16,26,189.0,1,0.0,3.0,14.0,1,1,252.0,88.0,10.0,5.523996,0,0.750000,0.113748,29,125.0,2,1.0,0.0,9.0,3.311308,0,16,25,189.0,1,0.0,0.0,252.0,88.0,10.0,10.052255,0,11.671907,0.0,58.959999,74.959999
6566,2023,18,TEN,28,1,QBWRWRRBTEWRTEPRB,17,27,168.0,2,1.0,2.0,16.0,0,0,143.0,123.0,10.0,0.463210,0,1.166667,0.083687,26,175.0,2,1.0,0.0,8.0,4.266917,0,17,26,168.0,2,0.0,0.0,152.0,123.0,10.0,3.145758,0,-3.029114,0.0,71.020004,88.020004
6567,2023,18,JAX,20,0,TEWRWRRBWRQBRBRBWR,29,43,280.0,2,2.0,0.0,0.0,0,0,308.0,109.0,12.0,1.649861,0,0.909091,0.067412,23,82.0,0,0.0,0.0,5.0,-5.792741,0,29,41,280.0,2,0.0,0.0,301.0,109.0,12.0,3.390518,0,1.255574,0.0,63.400002,92.400002
6568,2023,18,WAS,10,1,TEWRWRRBWRRBRBWRTEQBWRRB,20,28,157.0,1,2.0,4.0,27.0,0,0,154.0,146.0,9.0,-7.128528,0,5.000000,0.030534,17,50.0,0,1.0,1.0,3.0,-7.246222,0,20,28,157.0,1,0.0,0.0,154.0,146.0,9.0,-0.754869,0,5.506416,0.0,30.980000,50.980000


In [9]:
weeks['next_week_score'] = weeks.groupby(['team', 'season'])['score'].shift(-1)

In [10]:
data = []

for season in range(2021, 2024):
    for week_num in range(20):

        url = f"https://www.nfl.com/news/nfl-power-rankings-week-{week_num}-{season}-nfl-season"

        response = requests.get(url)
        status = response.status_code

        if status == 200:
            print("whoohoo")
            soup = BeautifulSoup(response.text, 'html.parser')

            for item in soup.find_all('div', class_='nfl-o-ranked-item'):
                # Extract the rank
                rank = item.find('div', class_='nfl-o-ranked-item__label--second').text.strip()
    
                # Extract the team name
                team = item.find('div', class_='nfl-o-ranked-item__title').text.strip()
    
                # Extract the record
                record = item.find('div', class_='nfl-o-ranked-item__info').span.text.strip()
    
                # Find the next sibling that contains the description
                description_div = item.find_next_sibling('div', class_='nfl-c-body-part--text')
                if description_div:
                    description = ' '.join(description_div.stripped_strings)
                else:
                    description = ''
    
                data.append({'season': season, 'week': week_num,
                             'rank': rank, 'team': team,
                             'record': record, 'description': description})
        else:
            print(status, season, week_num)

df = pd.DataFrame(data)
print(df)

404 2021 0
404 2021 1
404 2021 2
404 2021 3
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
404 2021 17
whoohoo
404 2021 19
404 2022 0
404 2022 1
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
404 2022 19
404 2023 0
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
whoohoo
404 2023 19
      season  week rank                   team record  \
0       2021     4    1       Los Angeles Rams    3-0   
1       2021     4    2   Tampa Bay Buccaneers    2-1   
2       2021     4    3      Green Bay Packers    2-1   
3       2021     4    4          Buffalo Bills    2-1   
4       2021     4    5       Baltimore Ravens    2-1   
...      ...   ...  ...                    ...    ...   
1563    2023    18   28        New York Giants   5-11   
1564    2023    18   29    

In [11]:
df = df.rename(columns={'rank': 'power_rank'})
df['team'] = df['team'].replace(name_id_dict)

df

,season,week,power_rank,team,record,description
0,2021,4,1,LAR,3-0,Previous rank: No. 4 The marriage between Sean...
1,2021,4,2,TB,2-1,Previous rank: No. 1 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
2,2021,4,3,GB,2-1,Previous rank: No. 6 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
3,2021,4,4,BUF,2-1,Previous rank: No. 5 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
4,2021,4,5,BAL,2-1,Previous rank: No. 3 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
...,...,...,...,...,...,...
1563,2023,18,28,NYG,5-11,The Giants sure are fascinating. There are amp...
1564,2023,18,29,ARI,4-12,"The Cardinals have relished the spoiler role, ..."
1565,2023,18,30,NE,4-12,We still have no clarity on Bill Belichick's s...
1566,2023,18,31,WAS,4-12,We've watched Sam Howell crest and fall multip...


In [12]:
def clean_power_rankings(df, column_name):
    # Define regular expressions to match patterns
    pattern_gibberish = r'»¿ï\|Previous Rank: \d+'
    nfl_teams = [
        'Cardinals', 'Falcons', 'Ravens', 'Bills', 'Panthers', 'Bears', 'Bengals',
        'Browns', 'Cowboys', 'Broncos', 'Lions', 'Packers', 'Texans', 'Colts',
        'Jaguars', 'Chiefs', 'Raiders', 'Chargers', 'Rams', 'Dolphins', 'Vikings',
        'Patriots', 'Saints', 'Giants', 'Jets', 'Eagles', 'Steelers', '49ers',
        'Seahawks', 'Buccaneers', 'Titans', 'Football Team', 'Commanders', 'Bucs'
    ]

    # Create a regular expression pattern to match NFL team names
    pattern_nfl_teams = r'\b(?:' + '|'.join(nfl_teams) + r')\b'

    # Remove matching patterns from the specified column
    df[column_name] = df[column_name].str.replace(pattern_gibberish, '', regex=True)
    df[column_name] = df[column_name].str.replace(pattern_nfl_teams, '', regex=True)

    return df

clean_power_rankings(df, 'description')


,season,week,power_rank,team,record,description
0,2021,4,1,LAR,3-0,Previous rank: No. 4 The marriage between Sean...
1,2021,4,2,TB,2-1,Previous rank: No. 1 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
2,2021,4,3,GB,2-1,Previous rank: No. 6 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
3,2021,4,4,BUF,2-1,Previous rank: No. 5 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
4,2021,4,5,BAL,2-1,Previous rank: No. 3 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...
...,...,...,...,...,...,...
1563,2023,18,28,NYG,5-11,The sure are fascinating. There are ample rea...
1564,2023,18,29,ARI,4-12,"The have relished the spoiler role, starting ..."
1565,2023,18,30,NE,4-12,We still have no clarity on Bill Belichick's s...
1566,2023,18,31,WAS,4-12,We've watched Sam Howell crest and fall multip...


In [13]:
df = df.merge(weeks, how='inner')
df

,season,week,power_rank,team,record,description,score,is_home,position,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,special_teams_tds,fantasy_points,fantasy_points_ppr,next_week_score
0,2021,4,2,TB,2-1,Previous rank: No. 1 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...,19,0,QBWRTEWRRBTEWRTERBWR,22,43,269.0,0,0.0,1.0,8.0,0,0,459.0,109.0,12.0,2.245208,0,0.586057,0.015135,30,120.0,1,0.0,0.0,8.0,4.894323,0,22,43,269.0,0,0.0,0.0,459.0,109.0,12.0,3.553649,0,-4.562038,0.0,55.660000,77.660004,45.0
1,2021,4,3,GB,2-1,Previous rank: No. 6 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...,27,1,QBTEWRWRRBTEWRWRRB,20,36,248.0,2,0.0,3.0,12.0,0,0,344.0,133.0,13.0,3.578705,0,0.720930,0.053233,33,131.0,1,1.0,1.0,9.0,-0.836024,0,20,35,248.0,2,0.0,0.0,322.0,133.0,13.0,9.241535,0,11.453997,0.0,71.820000,91.820000,25.0
2,2021,4,4,BUF,2-1,Previous rank: No. 5 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...,40,1,WRWRWRQBQBRBTEFBWRRB,21,31,256.0,2,1.0,1.0,5.0,0,0,271.0,73.0,12.0,2.466697,0,4.921933,0.144098,40,199.0,2,2.0,0.0,12.0,3.043254,0,21,31,256.0,2,1.0,0.0,271.0,73.0,12.0,3.501306,0,12.004599,0.0,85.739998,106.740005,38.0
3,2021,4,5,BAL,2-1,Previous rank: No. 3 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...,23,0,RBRBRBWRFBTEQBTEWRWRWR,22,37,316.0,1,0.0,3.0,12.0,0,0,420.0,97.0,14.0,10.418509,0,0.752381,0.132586,30,102.0,1,0.0,0.0,6.0,-3.287799,0,22,35,316.0,1,0.0,0.0,406.0,97.0,14.0,16.041042,0,5.378909,0.0,70.440002,92.440002,31.0
4,2021,4,6,KC,1-2,Previous rank: No. 2 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï...,42,0,TEFBWRQBWRRBWRTERBTE,24,30,278.0,5,1.0,1.0,7.0,0,0,219.0,117.0,16.0,22.732895,0,1.269406,0.351591,32,200.0,1,0.0,0.0,12.0,10.220598,0,24,30,278.0,5,0.0,0.0,219.0,117.0,16.0,24.253073,0,15.606176,0.0,112.919998,136.919998,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,2023,18,28,NYG,5-11,The sure are fascinating. There are ample rea...,27,1,QBTEWRRBWRWRWRRBQBWR,25,34,311.0,1,1.0,2.0,5.0,0,0,321.0,146.0,13.0,8.102219,0,3.739873,0.170319,30,109.0,2,1.0,0.0,9.0,-2.271896,0,25,32,311.0,1,0.0,0.0,310.0,146.0,13.0,13.034679,0,7.307635,0.0,74.440002,99.440002,NaN
1407,2023,18,29,ARI,4-12,"The have relished the spoiler role, starting ...",20,1,RBQBWRRBWRTEWRQBTE,22,30,262.0,1,0.0,1.0,2.0,0,0,167.0,171.0,11.0,14.697531,0,1.568862,0.195219,38,206.0,1,0.0,0.0,13.0,1.351517,0,22,29,262.0,1,0.0,0.0,170.0,171.0,11.0,15.790702,0,4.588209,0.0,73.279999,95.279999,NaN
1408,2023,18,30,NE,4-12,We still have no clarity on Bill Belichick's s...,3,1,WRRBTETEWRRBQBWR,12,30,88.0,0,2.0,7.0,57.0,0,0,319.0,52.0,3.0,-27.749020,0,0.275862,-0.064405,21,88.0,0,0.0,0.0,3.0,-4.700653,0,12,29,88.0,0,0.0,0.0,295.0,52.0,3.0,-15.649771,0,6.207394,0.0,17.120001,29.120001,NaN
1409,2023,18,31,WAS,4-12,We've watched Sam Howell crest and fall multip...,10,1,TEWRWRRBWRRBRBWRTEQBWRRB,20,28,157.0,1,2.0,4.0,27.0,0,0,154.0,146.0,9.0,-7.128528,0,5.000000,0.030534,17,50.0,0,1.0,1.0,3.0,-7.246222,0,20,28,157.0,1,0.0,0.0,154.0,146.0,9.0,-0.754869,0,5.506416,0.0,30.980000,50.980000,NaN


In [14]:
def add_score_averages(df, avg_columns):
    # Ensure the dataframe is sorted by season, team, and week for correct rolling calculations
    df = df.sort_values(by=['season', 'team', 'week'])

    for col in avg_columns:
        # Calculate the season average score for each team for the specified column
        df[f'{col}_season_avg'] = df.groupby(['season', 'team'])[col].transform('mean')

        # Calculate the rolling 5-week average score for each team for the specified column
        df[f'{col}_rolling_5_week_avg'] = df.groupby(['season', 'team'])[col].rolling(window=5, min_periods=1).mean().reset_index(level=[0,1], drop=True)

    return df

# Example usage:
# result_df = calculate_averages(df)


avg_cols = ['score', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
            'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
            'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions',
            'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
            'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
            'racr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']

result_df = add_score_averages(df, avg_cols)
result_df

,season,week,power_rank,team,record,description,score,is_home,position,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,special_teams_tds,fantasy_points,fantasy_points_ppr,next_week_score,score_season_avg,score_rolling_5_week_avg,completions_season_avg,completions_rolling_5_week_avg,attempts_season_avg,attempts_rolling_5_week_avg,passing_yards_season_avg,passing_yards_rolling_5_week_avg,passing_tds_season_avg,passing_tds_rolling_5_week_avg,interceptions_season_avg,interceptions_rolling_5_week_avg,sacks_season_avg,sacks_rolling_5_week_avg,sack_yards_season_avg,sack_yards_rolling_5_week_avg,sack_fumbles_season_avg,sack_fumbles_rolling_5_week_avg,sack_fumbles_lost_season_avg,sack_fumbles_lost_rolling_5_week_avg,passing_air_yards_season_avg,passing_air_yards_rolling_5_week_avg,passing_yards_after_catch_season_avg,passing_yards_after_catch_rolling_5_week_avg,passing_first_downs_season_avg,passing_first_downs_rolling_5_week_avg,passing_epa_season_avg,passing_epa_rolling_5_week_avg,passing_2pt_conversions_season_avg,passing_2pt_conversions_rolling_5_week_avg,pacr_season_avg,pacr_rolling_5_week_avg,dakota_season_avg,dakota_rolling_5_week_avg,carries_season_avg,carries_rolling_5_week_avg,rushing_yards_season_avg,rushing_yards_rolling_5_week_avg,rushing_tds_season_avg,rushing_tds_rolling_5_week_avg,rushing_fumbles_season_avg,rushing_fumbles_rolling_5_week_avg,rushing_fumbles_lost_season_avg,rushing_fumbles_lost_rolling_5_week_avg,rushing_first_downs_season_avg,rushing_first_downs_rolling_5_week_avg,rushing_epa_season_avg,rushing_epa_rolling_5_week_avg,rushing_2pt_conversions_season_avg,rushing_2pt_conversions_rolling_5_week_avg,racr_season_avg,racr_rolling_5_week_avg,special_teams_tds_season_avg,special_teams_tds_rolling_5_week_avg,fantasy_points_season_avg,fantasy_points_rolling_5_week_avg,fantasy_points_ppr_season_avg,fantasy_points_ppr_rolling_5_week_avg
9,2021,4,11,ARI,3-0,Previous rank: No. 10 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿...,37,0,QBWRWRTERBRBWRQBWR,24,32,268.0,2,0.0,3.0,19.0,0,0,230.0,140.0,11.0,13.925697,0,1.165217,0.234009,40,216.0,2,1.0,0.0,15.0,8.409242,0,24,29,268.0,2,0.0,0.0,196.0,140.0,11.0,20.778736,0,11.418978,0.0,91.120003,115.120003,17.0,24.692308,37.000000,23.846154,24.0,34.615385,32.00,256.000000,268.000000,1.384615,2.000000,0.538462,0.00,2.538461,3.000000,22.384615,19.00,0.461538,0.0,0.076923,0.0,248.846161,230.000000,129.230774,140.000000,12.230769,11.0,3.138380,13.925697,0.000000,0.0,1.470610,1.165217,0.123189,0.234009,29.230769,40.000000,124.538460,216.000000,1.384615,2.0,1.153846,1.000000,0.000000,0.0,7.538462,15.000000,-0.669162,8.409242,0.000000,0.0,-0.083349,11.418978,0.0,0.0,69.009232,91.120003,92.855385,115.120003
30,2021,5,1,ARI,4-0,Previous rank: No. 11 ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿ï»¿...,17,1,WRWRTERBRBWRQBWR,22,31,239.0,1,0.0,2.0,29.0,0,0,218.0,118.0,11.0,4.363636,0,1.096330,0.138608,27,94.0,1,2.0,0.0,8.0,-1.820860,0,22,28,239.0,1,1.0,1.0,217.0,118.0,11.0,7.001238,0,-5.774274,0.0,56.860001,78.860001,37.0,24.692308,27.000000,23.846154,23.0,34.615385,31.50,256.000000,253.500000,1.384615,1.500000,0.538462,0.00,2.538461,2.500000,22.384615,24.00,0.461538,0.0,0.076923,0.0,248.846161,224.000000,129.230774,129.000000,12.230769,11.0,3.138380,9.144667,0.000000,0.0,1.470610,1.130774,0.123189,0.186309,29.230769,33.500000,124.538460,155.000000,1.384615,1.5,1.153846,1.500000,0.000000,0.0,7.538462,11.500000,-0.669162,3.294191,0.000000,0.0,-0.083349,2.822352,0.0,0.0,69.009232,73.990002,92.855385,96.990002
61,2021,6,2,ARI,5-0,"Previous ran

In [19]:
def convert_types(result_df):
    for col in result_df.columns:
        if result_df[col].dtype == 'object':
            try:
                # Try converting object column to numeric first (to catch numerical strings)
                result_df[col] = pd.to_numeric(result_df[col], errors='raise')
            except ValueError:
                # If conversion to float fails, convert to categorical
                result_df[col] = result_df[col].astype('category')
        elif result_df[col].dtype in ['int64', 'float64']:
            # Check if all non-null values in a numeric column can be integers
            if pd.notnull(result_df[col]).all() and (result_df[col] % 1 == 0).all():
                result_df[col] = result_df[col].astype('int')
            else:
                result_df[col] = result_df[col].astype('float')
    return result_df

result_df = convert_types(result_df)

TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first

In [16]:
for col in result_df.select_dtypes(include=['category']).columns:
    if result_df[col].isnull().any():  # Check if there are any nulls in the column
        most_frequent = result_df[col].mode()[0]  # Find the mode of the column
        result_df[col] = result_df[col].fillna(most_frequent)

In [18]:
# Split the data into features and target variable
X = result_df.drop(columns=['next_week_score', 'description', 'position'])
y = result_df['next_week_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, enable_categorical=True)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(mae)

TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first

In [ ]:
# Feature importances
xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

# SHAP values - Explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# SHAP summary plot (beeswarm)
shap.summary_plot(shap_values, X_test)

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def text_to_embedding(text):
    # Tokenize the text and convert to input format expected by BERT
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get the embeddings from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the mean of the last hidden states as the sentence embedding, and convert to list for pandas compatibility
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()  # Convert to list
    return embeddings

text_arr = []

for i, text in enumerate(result_df['description']):
    text_embeddings = text_to_embedding(text)
    text_arr.append(text_embeddings)
    print(i)
# text_embeddings 

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=1, verbose=1, perplexity=40, n_iter=300)

# Fit and transform the embeddings
text_tsne = tsne.fit_transform(np.array(text_arr))


In [ ]:
result_df['text_tsne'] = text_tsne

In [ ]:
# Split the data into features and target variable
X = result_df.drop(columns=['next_week_score', 'description', 'position'])
y = result_df['next_week_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, enable_categorical=True)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(mae)

# Feature importances
xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

# SHAP values - Explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# SHAP summary plot (beeswarm)
shap.summary_plot(shap_values, X_test)

In [ ]:
result_df['pred_score'] = model.predict(X)
result_df['mae'] = abs(result_df['next_week_score'] - result_df['pred_score'])

result_df.sort_values(by='mae', ascending=False)[['season', 'week', 'team', 'next_week_score', 'pred_score', 'mae']]